### 评测流程：
 1. 最easy的流程：从laion6plus中抽取500张图片作为验证集 --> 使用inference生成对应的图像 --> 使用gen的脚本进行评测
 2. scale up 图片数量 和 使用stable diffusion生成的图像作为GT

每次评测的时候都要重新生成一次图片，生成的图片需要save到对应的文件夹下面， 然后我再来这边更改的话十分麻烦，我干脆把这个步骤跟生成合二为一即可！

后面更可以考虑跟评测合二为一;

再后面直接考虑跟训练合二为一； 

最希望的是，我从最上层想要做一件什么事情，修改一下代码和对应的参数，直接给一个实验名字，所有的训练日志，ckpt, 评测结果都存到一个对应的地方可以方便地查看，而后分析结果即可；


In [1]:
import json
import random
import shutil

n = 10000

#### samle 10000个图文对
path = '/share/project/yfl/codebase/nb/laion6plus/part_1.json'
with open(path, 'r') as fn:
    d = json.load(fn)


samples = random.sample(d, n)

## 把samples的图像搬到自己的文件夹

for smaple in samples:
    val_dir = './validation/gt_imgs/'
    source = smaple['image']
    # file_name = source.split('/')[-1]
    target = val_dir
    shutil.copy(source, target)
    
### save 500个图文对
save_path = './validation/gt.json'

with open(save_path, 'w') as fn:
    json.dump(samples, fn)

In [4]:
samples[0]

{'source_caption': 'The Warriors of Dick Fight Island (Front L to R): Vampyr, Pisao, Taling, Judah, Lolo, Blanc (Back L to R): Sicolenaga and Hart',
 'image': '/share/projset/baaishare/laion6plus/train-00005/00185/00185287.jpg'}

## 构建COCO 2017 valset

In [1]:
import json

path = '/share/project/yfl/database/annotations2017/captions_val2017.json'

with open(path, 'r') as fn:
    d = json.load(fn) 

In [2]:
d.keys()

dict_keys(['info', 'licenses', 'images', 'annotations'])

In [3]:
len(d['annotations'])

25014

In [4]:
import random

n = 10000

samples = random.sample(d['annotations'], n)

In [14]:
samples[0]

{'image_id': 562197,
 'id': 2696,
 'caption': 'Three pictures of plates with salads, including beans, fish and vegetables.'}

In [5]:
import shutil
import json

img_root = '/share/project/yfl/database/val2017/'
tgt_root= '/share/project/yfl/codebase/git/AltTools/Altdiffusion/misc/coco_evaluation/gt_imgs/'
result = []

for sample in samples:
    caption = sample['caption']
    image_source = img_root + str(sample['image_id']).zfill(12) + '.jpg'
    image_tgt = tgt_root + str(sample['image_id']).zfill(12) + '.jpg'
    shutil.copy(image_source, tgt_root)
    result.append(
        {
            'source_caption': caption,
            'image': image_tgt 
        }
    )        

with open('/share/project/yfl/codebase/git/AltTools/Altdiffusion/misc/coco_evaluation/gt.json', 'w') as fn:
    json.dump(result, fn)

In [1]:
import json

path = '/share/project/yfl/codebase/nb/laion6plus/slices_96/slice_0.json'

with open(path, 'r') as fn:
    d = json.load(fn)

In [2]:
d[0]

{'source_caption': "<i>Django Unchained</i>'s Second Trailer Released",
 'image': '/share/projset/baaishare/laion6plus/train-00002/00927/00927878.jpg',
 'target_caption': "<i>Django Unchained</i>'s Second Trailer Released",
 'target': 'fr_XX'}

In [3]:
import glob

img_root = '/share/project/yfl/database/val2017/*'
img_paths = glob.glob(img_root)

img_paths[0]

'/share/project/yfl/database/val2017/000000175438.jpg'

In [5]:
import shutil
tgt_root= '/share/project/yfl/codebase/git/AltTools/Altdiffusion/misc/coco_evaluation/gt_imgs/'
for img_path in img_paths:
    shutil.copy(img_path, tgt_root)      